In [ ]:
# Install required packages
!pip install --upgrade google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib markdown-it-py beautifulsoup4

# Import necessary libraries
import os  # For interacting with the operating system (e.g., file paths)
import pickle  # For serializing and deserializing Python objects (e.g., credentials)
from google.auth.transport.requests import Request  # For refreshing credentials
from google_auth_oauthlib.flow import InstalledAppFlow  # For OAuth 2.0 authentication
from googleapiclient.discovery import build  # For building Google Docs API service
from markdown_it import MarkdownIt  # For converting Markdown to HTML
from bs4 import BeautifulSoup  # For parsing HTML content
import re  # For regular expressions (e.g., matching @name mentions)
from google.colab import drive  # For mounting Google Drive in Colab
from google.colab import auth  # For authenticating in Colab
from oauth2client.client import GoogleCredentials  # For handling Google credentials

# Mount Google Drive to access credentials and token files
drive.mount('/content/drive')

# Define the scope for Google Docs API
SCOPES = ['https://www.googleapis.com/auth/documents']

def get_google_auth():
    """
    Authenticate with Google Docs API and return credentials.
    If credentials are not valid or expired, refresh or create new credentials.
    """
    creds = None  # Initialize credentials as None
    token_path = '/content/drive/MyDrive/Colab Notebooks/token.pickle'  # Path to store/load credentials
    if os.path.exists(token_path):  # Check if token file exists
        with open(token_path, 'rb') as token:  # Open token file in read-binary mode
            creds = pickle.load(token)  # Load credentials from the file
    if not creds or not creds.valid:  # Check if credentials are missing or invalid
        auth.authenticate_user()  # Authenticate the user in Colab
        creds = GoogleCredentials.get_application_default()  # Get default credentials
        with open(token_path, 'wb') as token:  # Open token file in write-binary mode
            pickle.dump(creds, token)  # Save credentials to the file
    return creds  # Return the credentials

def initialize_docs_api():
    """
    Initialize the Google Docs API service using authenticated credentials.
    """
    creds = get_google_auth()  # Get valid credentials
    service = build('docs', 'v1', credentials=creds)  # Build Google Docs API service
    return service

def create_google_doc(service):
    """
    Create a new Google Doc with a specified title and return its document ID.
    """
    doc = service.documents().create(body={"title": "Markdown to Google Docs"}).execute()
    return doc.get('documentId')  # Return the ID of the created document

def markdown_to_html(markdown_text):
    """
    Convert Markdown text to HTML using the markdown-it-py library.
    """
    md = MarkdownIt("commonmark").enable('table').enable('strikethrough')  # Configure Markdown parser
    return md.render(markdown_text)  # Convert Markdown to HTML

def process_list_items(element, requests, index, nesting_level=0):
    """
    Process list items (both ordered and unordered) and apply proper formatting.
    - `element`: The HTML element representing the list.
    - `requests`: List of Google Docs API requests to be executed.
    - `index`: Current index in the document where content is being inserted.
    - `nesting_level`: Level of nesting for indentation.
    """
    list_type = element.name  # Type of list (ul or ol)
    for li in element.find_all('li', recursive=False):  # Iterate through list items
        text = ''.join(child.string or '' for child in li.children if not (hasattr(child, 'name') and child.name in ['ul', 'ol']))
        text = text.strip()  # Extract and clean text from list item
        if text:
            if text.startswith('[ ]') or text.startswith('[x]'):  # Handle checkbox items
                is_checked = text.startswith('[x]')  # Check if checkbox is checked
                content = text[3:].strip()  # Extract content after checkbox
                requests.append({
                    'insertText': {
                        'location': {'index': index},
                        'text': content + '\n'  # Insert content into document
                    }
                })

                # Apply bold style to assignee mentions (@name)
                assignee_mentions = re.finditer(r'@\w+', content)
                for match in assignee_mentions:
                    start = index + match.start()
                    end = index + match.end()
                    requests.append({
                        'updateTextStyle': {
                            'range': {'startIndex': start, 'endIndex': end},
                            'textStyle': {'bold': True},  # Apply bold style
                            'fields': 'bold'
                        }
                    })

                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': index, 'endIndex': index + len(content)},
                        'bulletPreset': 'BULLET_CHECKBOX'  # Add checkbox bullet
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': index, 'endIndex': index + len(content)},
                        'paragraphStyle': {
                            'indentStart': {'magnitude': 36 * nesting_level, 'unit': 'PT'},  # Indent nested lists
                            'indentFirstLine': {'magnitude': 36 * nesting_level, 'unit': 'PT'}
                        },
                        'fields': 'indentStart,indentFirstLine'
                    }
                })
                index += len(content) + 1  # Update index for next insertion
            else:  # Handle regular list items
                requests.append({
                    'insertText': {
                        'location': {'index': index},
                        'text': text + '\n'  # Insert content into document
                    }
                })

                # Apply bold style to assignee mentions (@name)
                assignee_mentions = re.finditer(r'@\w+', text)
                for match in assignee_mentions:
                    start = index + match.start()
                    end = index + match.end()
                    requests.append({
                        'updateTextStyle': {
                            'range': {'startIndex': start, 'endIndex': end},
                            'textStyle': {'bold': True},  # Apply bold style
                            'fields': 'bold'
                        }
                    })

                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': index, 'endIndex': index + len(text)},
                        'bulletPreset': 'BULLET_ARROW_DIAMOND_DISC'  # Add regular bullet
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': index, 'endIndex': index + len(text)},
                        'paragraphStyle': {
                            'indentStart': {'magnitude': 36 * nesting_level, 'unit': 'PT'},  # Indent nested lists
                            'indentFirstLine': {'magnitude': 36 * nesting_level, 'unit': 'PT'}
                        },
                        'fields': 'indentStart,indentFirstLine'
                    }
                })
                index += len(text) + 1  # Update index for next insertion

        # Handle nested lists recursively
        nested_lists = li.find_all(['ul', 'ol'], recursive=False)
        for nested_list in nested_lists:
            index = process_list_items(nested_list, requests, index, nesting_level + 1)

    return index  # Return updated index

def html_to_google_docs(service, doc_id, html_content):
    """
    Convert HTML content to Google Docs format and apply formatting.
    - `service`: Google Docs API service instance.
    - `doc_id`: ID of the Google Doc to be updated.
    - `html_content`: HTML content to be converted.
    """
    soup = BeautifulSoup(html_content, 'html.parser')  # Parse HTML content
    requests = []  # List of Google Docs API requests
    index = 1  # Starting index for document content
    is_footer = False  # Flag to identify footer content

    for element in soup.children:  # Iterate through HTML elements
        if not hasattr(element, 'name'):  # Skip non-HTML elements (e.g., text nodes)
            continue

        if element.name == 'h1':  # Handle Heading 1
            text = element.text.strip()
            requests.append({
                'insertText': {'location': {'index': index}, 'text': text + '\n'}  # Insert text
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': index, 'endIndex': index + len(text)},
                    'paragraphStyle': {'namedStyleType': 'HEADING_1'},  # Apply Heading 1 style
                    'fields': 'namedStyleType'
                }
            })
            index += len(text) + 1  # Update index

        elif element.name == 'h2':  # Handle Heading 2
            text = element.text.strip()
            requests.append({
                'insertText': {'location': {'index': index}, 'text': text + '\n'}  # Insert text
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': index, 'endIndex': index + len(text)},
                    'paragraphStyle': {'namedStyleType': 'HEADING_2'},  # Apply Heading 2 style
                    'fields': 'namedStyleType'
                }
            })
            index += len(text) + 1  # Update index

        elif element.name == 'h3':  # Handle Heading 3
            text = element.text.strip()
            requests.append({
                'insertText': {'location': {'index': index}, 'text': text + '\n'}  # Insert text
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': index, 'endIndex': index + len(text)},
                    'paragraphStyle': {'namedStyleType': 'HEADING_3'},  # Apply Heading 3 style
                    'fields': 'namedStyleType'
                }
            })
            index += len(text) + 1  # Update index

        elif element.name == 'p':  # Handle Paragraphs
            text = element.text.strip()
            requests.append({
                'insertText': {'location': {'index': index}, 'text': text + '\n'}  # Insert text
            })

            # Apply bold style to assignee mentions (@name)
            assignee_mentions = re.finditer(r'@\w+', text)
            for match in assignee_mentions:
                start = index + match.start()
                end = index + match.end()
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': start, 'endIndex': end},
                        'textStyle': {'bold': True},  # Apply bold style
                        'fields': 'bold'
                    }
                })

            # Apply footer style if we're in the footer section
            if is_footer:
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': index, 'endIndex': index + len(text)},
                        'textStyle': {'italic': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0.5, 'green': 0.5, 'blue': 0.5}}}},  # Apply italic and gray color
                        'fields': 'italic,foregroundColor'
                    }
                })
            else:
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': index, 'endIndex': index + len(text)},
                        'textStyle': {'bold': True, 'italic': True},  # Apply bold and italic
                        'fields': 'bold,italic'
                    }
                })
            index += len(text) + 1  # Update index

        elif element.name in ['ul', 'ol']:  # Handle Lists
            index = process_list_items(element, requests, index)

        elif element.name == 'hr':  # Handle Horizontal Rule (footer separator)
            requests.append({
                'insertText': {'location': {'index': index}, 'text': '----------------------------------------\n'}  # Insert separator
            })
            index += len('----------------------------------------\n')  # Update index
            is_footer = True  # Mark footer section

    if requests:  # Execute all requests if any exist
        service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Example Markdown content
markdown_text = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

try:
    # Initialize Google Docs API service
    docs_service = initialize_docs_api()
    print("Successfully authenticated and initialized Google Docs API")

    # Create a new Google Doc
    doc_id = create_google_doc(docs_service)

    # Convert Markdown to HTML
    html_content = markdown_to_html(markdown_text)
    print(html_content)

    # Convert HTML to Google Docs format
    html_to_google_docs(docs_service, doc_id, html_content)

    # Print the link to the created Google Doc
    print(f"Google Doc created successfully. Access it here: https://docs.google.com/document/d/{doc_id}/edit")

except Exception as e:  # Handle errors gracefully
    print(f"Error: {e}")